In [25]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())

import matplotlib.pyplot as plt



In [16]:
import requests

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
res = requests.get(
    "https://api.nasdaq.com/api/quote/list-type/nasdaq100", headers=headers
)
print(res.status_code)
nasd100_con = pd.DataFrame(res.json()["data"]["data"]["rows"])
nasd100_con

200


,symbol,sector,companyName,marketCap,lastSalePrice,netChange,percentageChange,deltaIndicator
0,AAPL,,Apple Inc. Common Stock,"3,323,662,273,500",$215.75,$1.00,-0.46%,down
1,AMAT,,"Applied Materials, Inc. Common Stock","196,569,503,915",$235.804,$1.606,-0.68%,down
2,AMGN,,Amgen Inc. Common Stock,"166,836,553,559",$309.99,$1.02,-0.33%,down
3,CMCSA,,Comcast Corporation Class A Common Stock,"149,764,806,252",$38.28,$0.11,+0.29%,up
4,INTC,,Intel Corporation Common Stock,"131,285,880,000",$30.7592,$0.0808,-0.26%,down
...,...,...,...,...,...,...,...,...
97,WBD,,"Warner Bros. Discovery, Inc. Series A Common S...","17,397,225,126",$7.12,$0.02,+0.28%,up
98,GEHC,,GE HealthCare Technologies Inc. Common Stock,"34,810,049,040",$76.10,$0.16,-0.21%,down
99,LIN,,Linde plc Ordinary Shares,"206,181,283,972",$427.05,$1.89,-0.44%,down
100,ARM,,Arm Holdings plc American Depositary Shares,"165,235,692,839",$156.49,$2.34,-1.47%,down


In [ ]:
import yfinance as yf
# set the end date to today
end_date = datetime.today()
# set the start date to 5 years ago
start_date = end_date - timedelta(days=5 * 365)
print(start_date)

In [5]:
nasd100_df = pd.DataFrame()
nasd100_df["NDX"] = yf.download("^NDX", start=start_date, end=end_date)["Close"]
nasd100_df.tail()

[*********************100%%**********************]  1 of 1 completed


,NDX
Date,
2024-06-25,19701.130859
2024-06-26,19751.050781
2024-06-27,19789.029297
2024-06-28,19682.869141
2024-07-01,19812.220703


In [19]:
sp500 = pd.read_csv("snp500_constituents.csv")
sp500.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,TMUS,T-Mobile US,Communication Services,Wireless Telecommunication Services,"Bellevue, Washington",15/07/2019,1283699,1994
1,AWK,American Water Works,Utilities,Water Utilities,"Camden, New Jersey",04/03/2016,1410636,1886
2,CPAY,Corpay,Financials,Transaction & Payment Processing Services,"Atlanta, Georgia",20/06/2018,1175454,2000
3,FIS,Fidelity National Information Services,Financials,Transaction & Payment Processing Services,"Jacksonville, Florida",10/11/2006,1136893,1968
4,FI,Fiserv,Financials,Transaction & Payment Processing Services,"Brookfield, Wisconsin",02/04/2001,798354,1984


In [22]:
drop_dector = [
    "Financials",
    "Real Estates",
    "Information Technology",
    "Materials"
]

sp500 = sp500.loc[~sp500['GICS Sector'].isin(drop_dector)]
sp500 = sp500.loc[~sp500['Symbol'].isin(nasd100_con['symbol'].to_list())]

sp500.shape

(283, 8)

In [46]:
# for s in sp500['Symbol'].to_list():
#     try:
#         close_price = yf.download(s, start=start_date, end=end_date)["Close"]
#         close_price = pd.merge(close_price, nasd100_df, left_index=True, right_index=True)
#         covariance.loc[s] = [close_price['Close'].cov(close_price['NDX'])]
#     except:
#         continue

close_price = yf.download(sp500['Symbol'].to_list(), start=start_date, end=end_date)["Close"]
close_price = pd.merge(close_price, nasd100_df, left_index=True, right_index=True)

[**********************75%%**********            ]  213 of 283 completed

$BF.B: possibly delisted; No price data found  (1d 2019-07-04 14:31:27.583265 -> 2024-07-02 14:31:27.583265)


[*********************100%%**********************]  283 of 283 completed

1 Failed download:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2019-07-04 14:31:27.583265 -> 2024-07-02 14:31:27.583265)')


In [56]:
covariance = {}

for s in close_price.columns:
    covariance[s] = close_price[s].cov(close_price["NDX"])

covariance_df = pd.Series(covariance).to_frame()
covariance_df = covariance_df.reset_index()
covariance_df.columns = ['symbol', 'cov']
covariance_df

,symbol,cov
0,A,6.049461e+04
1,AAL,-6.166766e+03
2,ABBV,6.780990e+04
3,ABT,2.825583e+04
4,ADM,2.257467e+04
...,...,...
279,XYL,4.842748e+04
280,YUM,3.485050e+04
281,ZBH,-2.099576e+02
282,ZTS,6.408792e+04


In [57]:
covariance_df.sort_values(['cov'])

,symbol,cov
242,TFX,-7.393901e+04
20,BA,-6.340392e+04
33,BXP,-3.687090e+04
21,BAX,-3.288074e+04
174,MMM,-2.428789e+04
...,...,...
181,MTD,5.987068e+05
19,AZO,1.400468e+06
190,NVR,3.336076e+06
283,NDX,8.905839e+06


In [63]:
def get_market_cap(symbol):
    try: 
        market_cap = yf.Ticker(symbol).fast_info['market_cap']
        market_cap = market_cap / 10**6
    except:
        market_cap = np.nan
    return market_cap

covariance_df['market_cap'] = covariance_df['symbol'].apply(get_market_cap)
covariance_df.head()

GEV: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
$ROK: possibly delisted; No price data found  (period=1y)
$SO: possibly delisted; No price data found  (period=1y)
SOLV: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
$WYNN: possibly delisted; No price data found  (period=1y)


,symbol,cov,market_cap
0,A,60494.610050,37091.574112
1,AAL,-6166.766061,7249.794448
2,ABBV,67809.895046,297778.658628
3,ABT,28255.825003,179529.805737
4,ADM,22574.665653,30051.942009


In [72]:
covariance_df = covariance_df.dropna(how='any').sort_values(['market_cap'], ascending=False)
covariance_df[covariance_df['cov'] < 0].to_csv('./sp500_biggest_neg_cov.csv')